# Jane Street Puzzle 04-2021: Bracketology 101

# Theory

Let $p_{ij}$ be the probability that $i$-seed wins against $j$-seed and $q_{ir}$ the probability that seed $i$ wins in round $r$ (there are 4 rounds in total). Based on the bracket, we can define the set $O_{ir}$ of possible opponents for the $i$-seed in round $r$ with $O_{i,1}$ being the singleton containing the $i$-seed's first opponent.

Thus, $q$ can be otained as follows:

$$q_{ir}=\begin{cases}p_{io},\, o\in O_{i,r} &\text{if } r=1\\
q_{i,r-1} \cdot \displaystyle\sum_{o\in O_{i,r}}  p_{io} \cdot q_{o, r-1} & \text{else}
\end{cases}$$

We are interested in $q_{2,4}$, the probability that the $2$-seed wins the tournament.

## Computational Implementation

Parameters and Variables

In [1]:
probs = dict()

for i in range(1, 17):
    for j in range(i + 1, 17):
        probs[(i, j)] = j / (i + j)
        probs[(j, i)] = 1 - probs[(i,j)]

default_bracket = [1, 16, 8, 9, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15]

In [2]:
def prob_wins_r(i, n, bracket):
    """returns the probability that seed i wins round n given a bracket"""
    r_index = bracket.index(i) // 2**n
    opp = r_index + 1 if r_index % 2 == 0 else r_index - 1

    if n == 0:
        return probs[(i, bracket[opp])]

    prob = 0
    for j in range(len(bracket)):
        if j // 2**n == opp:
            prob += probs[(i, bracket[j])] * prob_wins_r(bracket[j], n - 1, bracket)

    return prob_wins_r(i, n - 1, bracket) * prob

In [3]:
def best_change(bracket):
    best_change = None
    best_prob = prob_wins_r(2, 3, bracket)

    for i in range(1, 9):
        for j in range(i + 1, 17):
            i_index = bracket.index(i)
            j_index = bracket.index(j)

            new_bracket = bracket.copy()

            new_bracket[i_index] = j
            new_bracket[j_index] = i 

            prob_2_wins = prob_wins_r(2, 3, new_bracket)
            if prob_2_wins > best_prob:
                best_prob = prob_2_wins
                best_change = (i, j)
    
    return best_change, best_prob

In [4]:
prob_2_wins = prob_wins_r(2, 3, default_bracket)
bst_change, best_prob = best_change(default_bracket)

print(f"Best result achieved when changing {bst_change[0]} with {bst_change[1]} with a difference in the probability of winning of {(best_prob - prob_2_wins) * 100:.5f}%")

Best result achieved when changing 3 with 16 with a difference in the probability of winning of 6.55795%
